In [19]:
import os
from pprint import pprint

from opa.http_methods import get_json_data
from opa.env import get_secret

os.putenv("USE_HTTP_CACHE", "True")
ticker = "AAPL"
fmp_key = get_secret("fmp_cloud_api_key")
alphavantage_key = get_secret("alphavantage_api_key")

In [25]:
def print_data_span(all_dates: list[any]):
    pprint(f'Data spans from {min(all_dates)} to {max(all_dates)}')

# Market providers overview

There are various market providers available.

## FMP Cloud

A full history can be retrieved using this API

In [26]:
fmp_full_history = get_json_data(
    f"https://fmpcloud.io/api/v3/historical-price-full/{ticker}",
    params={"serietype": "line", "apikey": fmp_key},
)
print_data_span([v["date"] for v in fmp_full_history["historical"]])
pprint({
    "symbol": fmp_full_history["symbol"],
    "historical": fmp_full_history["historical"][:5]
})

'Data spans from 1980-12-12 to 2023-07-14'
{'historical': [{'close': 190.69, 'date': '2023-07-14'},
                {'close': 190.54, 'date': '2023-07-13'},
                {'close': 189.77, 'date': '2023-07-12'},
                {'close': 188.08, 'date': '2023-07-11'},
                {'close': 188.61, 'date': '2023-07-10'}],
 'symbol': 'AAPL'}


A shorter but more complete history can be retrieved like this : 

In [27]:
fmp_shorter_history = get_json_data(
    f"https://fmpcloud.io/api/v3/historical-price-full/{ticker}",
    params={"apikey": fmp_key},
)

print_data_span([v["date"] for v in fmp_shorter_history["historical"]])

pprint({
    "symbol": fmp_shorter_history["symbol"],
    "historical": fmp_shorter_history["historical"][:2]
})

'Data spans from 2018-07-18 to 2023-07-14'
{'historical': [{'adjClose': 190.690002,
                 'change': 0.46,
                 'changeOverTime': 0.0024181,
                 'changePercent': 0.24181,
                 'close': 190.69,
                 'date': '2023-07-14',
                 'high': 191.1799,
                 'label': 'July 14, 23',
                 'low': 189.63,
                 'open': 190.23,
                 'unadjustedVolume': 40764621,
                 'volume': 40835691,
                 'vwap': 190.46},
                {'adjClose': 190.539993,
                 'change': 0.04,
                 'changeOverTime': 0.0002099738,
                 'changePercent': 0.02099738,
                 'close': 190.54,
                 'date': '2023-07-13',
                 'high': 191.19,
                 'label': 'July 13, 23',
                 'low': 189.78,
                 'open': 190.5,
                 'unadjustedVolume': 41342300,
                 'volume': 41337338

In [28]:
fmp_streaming = get_json_data(
    f"https://fmpcloud.io/api/v3/historical-chart/15min/{ticker}",
    params={"apikey": fmp_key},
)

print_data_span([v["date"] for v in fmp_streaming])
pprint(fmp_streaming[:4])

'Data spans from 2023-05-17 09:30:00 to 2023-07-14 16:00:00'
[{'close': 190.69,
  'date': '2023-07-14 16:00:00',
  'high': 190.71,
  'low': 190.3978,
  'open': 190.69,
  'volume': 1654688},
 {'close': 190.72,
  'date': '2023-07-14 15:45:00',
  'high': 190.74,
  'low': 190.26,
  'open': 190.42,
  'volume': 3749214},
 {'close': 190.415,
  'date': '2023-07-14 15:30:00',
  'high': 190.49,
  'low': 190.16,
  'open': 190.275,
  'volume': 1254758},
 {'close': 190.275,
  'date': '2023-07-14 15:15:00',
  'high': 190.42,
  'low': 190.04,
  'open': 190.04,
  'volume': 1315560}]


## Alphavantage

The cell below displays the kind of data one can get as full history

In [30]:

alpha_full_history = get_json_data(
    f"https://www.alphavantage.co/query",
    params={
        "function": "TIME_SERIES_DAILY_ADJUSTED",
        "symbol": ticker,
        "apikey": alphavantage_key,
    },
)
print_data_span(alpha_full_history["Time Series (Daily)"].keys())
pprint({
    "Meta Data": alpha_full_history["Meta Data"],
    "Time Series (Daily)": dict(list(alpha_full_history["Time Series (Daily)"].items())[:3])
})


'Data spans from 2023-02-21 to 2023-07-14'
{'Meta Data': {'1. Information': 'Daily Time Series with Splits and Dividend '
                                 'Events',
               '2. Symbol': 'AAPL',
               '3. Last Refreshed': '2023-07-14',
               '4. Output Size': 'Compact',
               '5. Time Zone': 'US/Eastern'},
 'Time Series (Daily)': {'2023-07-12': {'1. open': '189.68',
                                        '2. high': '191.7',
                                        '3. low': '188.47',
                                        '4. close': '189.77',
                                        '5. adjusted close': '189.77',
                                        '6. volume': '60750248',
                                        '7. dividend amount': '0.0000',
                                        '8. split coefficient': '1.0'},
                         '2023-07-13': {'1. open': '190.5',
                                        '2. high': '191.19',
               

The cell below displays the streaming data.

In [32]:
alpha_streaming = get_json_data(
    f"https://www.alphavantage.co/query",
    params={
        "function": "TIME_SERIES_INTRADAY",
        "symbol": ticker,
        "interval": "15min",
        "apikey": alphavantage_key,
    },
)

print_data_span(alpha_streaming["Time Series (15min)"].keys())
pprint({
    "Meta Data": alpha_streaming["Meta Data"],
    "Time Series (15min)": dict(list(alpha_streaming["Time Series (15min)"].items())[:3])
})

'Data spans from 2023-07-13 11:00:00 to 2023-07-14 19:45:00'
{'Meta Data': {'1. Information': 'Intraday (15min) open, high, low, close '
                                 'prices and volume',
               '2. Symbol': 'AAPL',
               '3. Last Refreshed': '2023-07-14 19:45:00',
               '4. Interval': '15min',
               '5. Output Size': 'Compact',
               '6. Time Zone': 'US/Eastern'},
 'Time Series (15min)': {'2023-07-14 19:15:00': {'1. open': '190.8000',
                                                 '2. high': '190.8100',
                                                 '3. low': '190.7500',
                                                 '4. close': '190.7600',
                                                 '5. volume': '2896'},
                         '2023-07-14 19:30:00': {'1. open': '190.7600',
                                                 '2. high': '190.7600',
                                                 '3. low': '190.7000',
           